In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split

In [2]:
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

In [68]:
split_word = [x.split(' ') for x in list(train_data['text'])]
split_word_test = [x.split(' ') for x in list(test_data['text'])]
train_data['text_split'] = split_word
test_data['text_split'] = split_word_test

In [71]:
train, valid = train_test_split(train_data, test_size = 0.2, random_state = 42) 

In [82]:
train.to_csv('data/train_use.csv', index=False)
valid.to_csv('data/valid_use.csv', index=False)
test_data.to_csv('data/test_use.csv', index=False)

## Sklearn 

In [75]:
train = pd.read_csv("data/train_use.csv")
valid = pd.read_csv("data/valid_use.csv")
test = pd.read_csv("data/test_use.csv")

In [76]:
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=5000)

In [77]:
X_raw = [str(train['keyword'][i]) + ' ' + train['text'][i] for i in range(len(train['keyword']))]
X_raw_valid = [str(valid['keyword'][i]) + ' ' + valid['text'][i] for i in range(len(valid['keyword']))]
X_raw_test = [str(test['keyword'][i]) + ' ' + test['text'][i] for i in range(len(test['keyword']))]
print(len(X_raw))
print(len(X_raw_valid))
print(len(X_raw_test))
print(len(X_raw) + len(X_raw_valid) + len(X_raw_test))

6090
1523
3263
10876


In [79]:
'''
t = pd.DataFrame()
t['words'] = X_raw
t['target'] = list(train['target'])
v = pd.DataFrame()
v['words'] = X_raw_valid
v['target'] = list(valid['target'])
te = pd.DataFrame()
te['words'] = X_raw_test
t.to_csv('data/train_rnn.csv', index=False)
v.to_csv('data/valid_rnn.csv', index=False)
te.to_csv('data/test_rnn.csv', index=False)
'''

"\nt = pd.DataFrame()\nt['words'] = X_raw\nt['target'] = list(train['target'])\nv = pd.DataFrame()\nv['words'] = X_raw_valid\nv['target'] = list(valid['target'])\nte = pd.DataFrame()\nte['words'] = X_raw_test\nt.to_csv('data/train_rnn.csv', index=False)\nv.to_csv('data/valid_rnn.csv', index=False)\nte.to_csv('data/test_rnn.csv', index=False)\n"

In [22]:
X_whole = X_raw.copy()
X_whole.extend(X_raw_valid)
X_whole.extend(X_raw_test)
print(len(X_whole))

10876


In [23]:
X = hv.transform(X_whole).toarray()
X.shape

(10876, 5000)

In [24]:
X_train = X[:6090]
X_valid = X[6090:(6090+1523)]
X_test = X[(6090+1523):]

In [74]:
y_train = list(train['target'])
y_valid = list(valid['target'])

In [ ]:
'''
X_train_pd = pd.DataFrame(X_train)
X_train_pd['target'] = y_train
X_train_pd.to_csv('data/train_lr.csv', index=False)
X_valid_pd = pd.DataFrame(X_valid)
X_valid_pd['target'] = y_valid
X_valid_pd.to_csv('data/valid_lr.csv', index=False)
X_test_pd = pd.DataFrame(X_test)
X_test_pd.to_csv('data/test_lr.csv', index=False)
'''

In [10]:
from sklearn.linear_model import LogisticRegression

In [26]:
lr = LogisticRegression(random_state=0).fit(X_train, y_train)
lr.score(X_valid, y_valid)

0.7839789888378201

In [27]:
lr_predict = lr.predict(X_test)
len(lr_predict)

3263

In [41]:
preds_df = pd.DataFrame({"id": test_data["id"], "target": lr_predict})
preds_df.to_csv(f"data/output_lr_15000.csv", index=False)